# UIUC Reboot Capstone Project

## Overview

### Loading Data

In [14]:
import pandas as pd
import glob as glob
import zipfile
import numpy as np               # to get unique values from list using numpy.unique 


In [56]:
# Read in files here

print("Compiling Product Reviews:\t\t", end='')

zf = zipfile.ZipFile('reviews.zip')
datReviews = [filename for filename in zf.namelist() if filename.endswith(".dat")]
print(len(datReviews))


Compiling Product Reviews:		50000


In [3]:
# Basic dataframe info